## Downstream task #########

In [ ]:
import sys
import os
import warnings
import glob
warnings.filterwarnings('ignore')  # ignore warnings, like ZeroDivision

# Go up one directory from 'notebooks/' to project root
project_root = os.path.abspath('..')
sys.path.append(project_root)

os.getcwd()

In [ ]:
import torch

# Check if GPU is available
if torch.cuda.is_available():
    # Set the device to GPU
    device = torch.device("cuda")
    print("GPU is available. Using GPU for computation.")
else:
    # If GPU is not available, fall back to CPU
    device = torch.device("cpu")
    print("GPU is not available. Using CPU for computation.")

In [ ]:
from src.transformation_utils import *
from src.utils_all import *
from src.utils_data import *

from src.MAE.utils_mae import *
from src.MAE.trainer_mae import *
from src.MAE.MAE_1D import *
from src.MAE.multi_trait import *
from src.MAE.trainer_trait import *

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import gc

##################
d = 10
h = 4

path_model_mae = '/home/mila/e/eya.cherif/scratch/mae/checkpoints_MAE_250304_0429_Trainingd{}_h{}_allUNlabels_155/best_model.h5'.format(d,h) #'MAE/mae_110225_Spectraratio75_spS20_Emb128_0cos500epochs_80all.h5' mae_110225_Spectraratio75_spS20_Emb128_0cos500epochs_100Ecosis
path_data_lb = '/home/mila/e/eya.cherif/Gan_project_test/49_all_lb_prosailPro.csv'
ls_tr = ["cab", "cw", "cm", "LAI", "cp", "cbc", "car", "anth"]

percentage_tr = 1
batch_size = 256


settings_dict_mae = {
    'learning_rate': 1e-3,
    'weight_decay': 0,
    'load_model_path': path_model_mae, #path_model_mae load_model_path
    'should_save_models': True,
    'skip_completed_experiment': True,
    'number_of_data_workers': 0,
    
    'mask_ratio': 0.75,
    'n_bands': 1720,
    'seq_size': 20,
    'in_chans': 1,
    'embed_dim': 128,
    'depth': d,
    'num_heads': h,
    'decoder_embed_dim': 128,
    'decoder_depth': 6,
    'decoder_num_heads': 4,
    'mlp_ratio': 4.0,
    'norm_layer': nn.LayerNorm,
    'cls_token': False,
    'device': torch.device("cuda" if torch.cuda.is_available() else "cpu")
}


sets = Settings()
sets.update_from_dict(settings_dict_mae)

test_mae = Trainer(sets)

test_mae.model_setup()
test_mae.model = torch.load(settings_dict_mae['load_model_path']) 


########## Trait regression ####
################ Data ###############
db_lb_all = pd.read_csv(path_data_lb, low_memory=False).drop(['Unnamed: 0'], axis=1)   

### external
groups = db_lb_all.groupby('dataset')

val_ext_idx = list(groups.get_group(32).index)+list(groups.get_group(3).index)
samples_val_ext = db_lb_all.loc[val_ext_idx,:]
db_lb_all.drop(val_ext_idx, inplace=True)

X_labeled, y_labeled, _ = data_prep_db(db_lb_all, ls_tr, weight_sample=True)
metadata = db_lb_all.iloc[:, :8]  # The metadata (dataset of origin)


red_ed = X_labeled.loc[:,750]
red_end = X_labeled.loc[:,1300]
red1000_ = X_labeled.loc[:,1000]

idx = X_labeled[(red_end>red1000_) & (red_ed>red1000_)].index

if(len(idx)>0):
    # X_labeled.loc[idx,:].T.plot(legend=False)
    X_labeled.drop(idx, inplace=True)
    y_labeled.drop(idx, inplace=True)
    metadata.drop(idx, inplace=True)


# Split labeled data into train (80%), validation (10%), and test (10%)
fr_sup, val_x= train_test_split(X_labeled, test_size=0.2, stratify=metadata.dataset, random_state=300)

y_sup = y_labeled.loc[fr_sup.index,:]
meta_train = metadata.loc[fr_sup.index,:]

val_y = y_labeled.loc[val_x.index,:]
meta_val = metadata.loc[val_x.index,:]

if(percentage_tr<1):
    fr_sup, _= train_test_split(fr_sup, test_size=1-percentage_tr, stratify=meta_train.dataset, random_state=300)
    
    y_sup = y_sup.loc[fr_sup.index,:]
    meta_train = meta_train.loc[fr_sup.index,:]

# ########## Scaler ###
scaler_list = None
scaler_model = save_scaler(y_sup, standardize=True, scale=True)

###### Create the dataset#################

train_dataset = SpectraDataset(fr_sup, y_sup, meta_train, augmentation=True, aug_prob=0.6)
# Define DataLoader with the custom collate function for fair upsampling
train_dataset_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#####
test_dataset = SpectraDataset(X_train=val_x, y_train=val_y, meta_train=meta_val, augmentation=False)
# Create DataLoader for the test dataset
valid_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

###########
# DB50_all_F_SHIFT.csv
path_data_lb_ext = '/home/mila/e/eya.cherif/Gan_project_test/DB50_all_F_SHIFT.csv'
db_lb_all_ext = pd.read_csv(path_data_lb_ext, low_memory=False).drop(['Unnamed: 0'], axis=1)   

ext_all = pd.concat([samples_val_ext,db_lb_all_ext])

ext_val_x = feature_preparation(ext_all.loc[:, '400':'2500']).loc[:, 400:2450] #ext_all samples_val_ext
ext_val_y = ext_all[ls_tr] # ext_all  samples_val_ext

# Validation dataset
x_p_val = torch.tensor(ext_val_x.values, dtype=torch.float)#.unsqueeze(dim=1) ##ext_val_x.values scaler_spectra.transform(ext_val_x.iloc[:,:-1])
lb_p_val = torch.tensor(ext_val_y.values,dtype=torch.float)

external_dataset = TensorDataset(x_p_val, lb_p_val) # x_p_val
valid_ext_loader = DataLoader(external_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
import torch

# # Load your model (example)
# model = torch.load("model.pth")
# model.eval()  # If you are evaluating the model

# Calculate the number of trainable parameters
num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Number of trainable parameters: {num_trainable_params}")

In [ ]:
fr_sup.shape

In [ ]:
plt.rc('font', size=10) #5.7
plt.rcParams['lines.markersize'] = 3
plt.rcParams['lines.linewidth'] = 2

images, lb, _ = next(iter(valid_loader))
x = images
x = x.to(device).view(x.shape[0], x.shape[-1])[:,:-1]

test_mae.model.eval()

with torch.no_grad():
    loss, pred, mask, z = test_mae.model(x) #, mask_ratio=0.75

ind = 100

spectra = pred.cpu().reshape(shape=(images.shape[0], -1))
pd.DataFrame([np.array(x[ind].cpu()),np.array(spectra[ind].cpu())]).T.plot()

# pd.DataFrame(scaler_spectra.inverse_transform(spectra.squeeze().detach().cpu().numpy())).T.plot(legend=False)

In [ ]:
images, lb, _ = next(iter(valid_loader))

test_mae.model.eval()
with torch.no_grad():
    x = images[:,:-1]
    x = x.to(device).view(x.shape[0], x.shape[-1])

    # embed patches
    x = test_mae.model.seq_embed(x)

    # add pos embed w/o cls token
    x = x + test_mae.model.pos_embed[:, np.sum(test_mae.model.is_cls_token):, :]
    
    x, mask, ids_restore = test_mae.model.half_masking(x)
    # x, mask, ids_restore = model.random_masking(x, 0.75)

    # apply Transformer blocks
    for blk in test_mae.model.blocks:
        x = blk(x)
    z = test_mae.model.norm(x)
    pred = test_mae.model.forward_decoder(z, ids_restore)


ind = 155
spectra = pred.cpu().reshape(shape=(images.shape[0], -1))
spectra.cpu().shape
np.array(images[ind].cpu())
pd.DataFrame([np.array(images[ind].cpu()), np.array(spectra[ind].cpu())]).T.plot()

In [ ]:
settings_dict = {
        'epochs': 50,
        'train_loader': train_dataset_loader,
        'valid_loader': valid_loader,
        # 'checkpoint_dir': checkpoint_dir,
        'batch_size': batch_size,
        'learning_rate': 1e-3,
        'weight_decay': 1e-4,
        'pretrained_model': test_mae.model,#model,
        'freeze': False,
        'early_stop': False,
        'patience': 10,
        'scaler_model': scaler_model,
    }

##########

# SEED = 155
# Create an empty dictionary
eval_scores = {}
ext_scores = {}

for SEED in [155]: #, 381, 187

    # Python random seed
    random.seed(SEED)
    
    # Numpy seed
    np.random.seed(SEED)
    
    # PyTorch seed
    torch.manual_seed(SEED)
    
    # If using CUDA
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)  # If using multi-GPU
    
    # Ensuring deterministic behavior
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    sets = Settings_reg()
    sets.update_from_dict(settings_dict)
    
    test_reg = Trainer_MAE_Reg(sets)
    
    test_reg.dataset_setup()
    
    ############ Moel modif ####
    lb_tr = len(ls_tr)
    # Hyperparameters
    output_dim = lb_tr  # Number of regression targets
    aggregation = "mean"  # No aggregation, use full sequence
    
    
    pretrained_model = test_mae.model #model
    latent_dim = pretrained_model.blocks[3].mlp.fc2.out_features #pretrained_model.patch_to_embedding.out_features #pretrained_model.blocks[3].mlp.fc2.out_features  # Feature size per token
    seq_size = (1720 // 20)  # Number of sequences in MAE (1720 // 10)
    
    
    # Instantiate the model
    test_reg.model = LatentRegressionModel(
        pretrained_encoder=pretrained_model,
        latent_dim=latent_dim,
        output_dim=output_dim,
        hidden_dims=None,  # None or hidden_dims Add dense layers
        aggregation=aggregation,
        normalize_latent=True,  # Enable latent normalization
    )
    
    ############
    test_reg.transformation_setup()
    test_reg.criterion_setup()
    
    test_reg.prepare_optimizers()
    test_reg.gpu_mode()
    
    for param in test_reg.model.encoder.parameters(): #encoder
        param.requires_grad = False
    
    test_reg.train_loop(epoch_start=1, num_epochs=200)
    
    ########### validation #######
    test_tr = ls_tr #list(val_y.columns)

    outputs_val = []
    lb_val = []
    
    for labeled_examples, labels, ds in tqdm(valid_loader):
        test_reg.model.eval()
        
        with torch.no_grad():
            labeled_examples = labeled_examples.to(device)[:,:-1]
            labels = labels.to(device)
            output_val = test_reg.model(labeled_examples.squeeze().float()) #[:,:-1]
            
            if(scaler_list is not None):
                outputs_val.append(scaler_list.inverse_transform(output_val.cpu().numpy()))
                lb_val.append(scaler_list.inverse_transform(labels.cpu().numpy()))
            
            if(test_reg.transformation_layer_inv is not None):  
                output_val = test_reg.transformation_layer_inv(output_val)
             
            outputs_val.append(output_val.cpu().numpy())
            lb_val.append(labels.cpu().numpy())
    
    pred = pd.DataFrame(np.concatenate(outputs_val, axis=0), columns=test_tr)#.shape
    obs_pf = pd.DataFrame(np.concatenate(lb_val, axis=0), columns=test_tr)
    
    val_mertics = eval_metrics(obs_pf, pred)
    eval_scores[SEED] = val_mertics
    ############## trans ####
    outputs_val = []
    lb_val = []
    
    for labeled_examples, labels in tqdm(valid_ext_loader):
        test_reg.model.eval()
        
        with torch.no_grad():
            labeled_examples = labeled_examples.to(device)[:,:-1]
            labels = labels.to(device)
            output_val = test_reg.model(labeled_examples.squeeze().float()) #[:,:-1]
            
            if(scaler_list is not None):
                outputs_val.append(scaler_list.inverse_transform(output_val.cpu().numpy()))
                lb_val.append(scaler_list.inverse_transform(labels.cpu().numpy()))
            
            if(test_reg.transformation_layer_inv is not None):  
                output_val = test_reg.transformation_layer_inv(output_val)
             
            outputs_val.append(output_val.cpu().numpy())
            lb_val.append(labels.cpu().numpy())
    
    pred = pd.DataFrame(np.concatenate(outputs_val, axis=0), columns=test_tr)#.shape
    obs_pf = pd.DataFrame(np.concatenate(lb_val, axis=0), columns=test_tr)
    
    val_mertics = eval_metrics(obs_pf, pred)
    ext_scores[SEED] = val_mertics
    
    del test_reg
    gc.collect()
    torch.cuda.empty_cache()

In [ ]:
# import pandas as pd

# # Load the CSV file
# df = pd.read_csv('mae_110225_Spectraratio75_spS20_Emb128_0cos500epochs_20all_seedsRun_LinearProbing_HalflRange_val.csv')  # Replace with your actual CSV filename

# # Reset index to treat 'Key' as a column and preserve the second level (cab, cw, etc.)
# # df.reset_index(inplace=True)
# df.rename(columns={'Unnamed: 0': 'Key'}, inplace=True)  # Rename the Key column if necessary
# df.rename(columns={'Unnamed: 1': 'index'}, inplace=True)  # Rename the Key column if necessary

# # Ensure 'Key' is numeric for grouping
# df["Key"] = pd.to_numeric(df["Key"], errors="coerce")  # Converts to numeric, ignoring errors

# # Set the correct hierarchical index (Key, Variable names like 'cab', 'cw', etc.)
# df.set_index(["Key", "index"], inplace=True)

# # Compute mean and standard deviation
# mean_metrics = df.groupby(level=1, sort=False).mean()
# std_metrics = df.groupby(level=1, sort=False).std(ddof=0) ## STDS ddof=1


# std_metrics

# # mean_metrics

In [ ]:
# Concatenate all DataFrames along a new axis and calculate mean and std
all_dfs = pd.concat(eval_scores.values(), keys=eval_scores.keys())  #eval_scores ext_scores

# Compute average and standard deviation across the dictionary
# Compute mean and standard deviation while keeping the original order
mean_metrics = all_dfs.groupby(level=1, sort=False).mean()
std_metrics = all_dfs.groupby(level=1, sort=False).std(ddof=0) ## STDS ddof=1

In [ ]:
mean_metrics

In [ ]:
mean_metrics.quantile(0.7)

In [ ]:
std_metrics

In [ ]:
# Concatenate all DataFrames along a new axis and calculate mean and std
all_dfs_ext = pd.concat(ext_scores.values(), keys=ext_scores.keys())  #eval_scores ext_scores

# Compute average and standard deviation across the dictionary
# Compute mean and standard deviation while keeping the original order
mean_metrics = all_dfs_ext.groupby(level=1, sort=False).mean()
std_metrics = all_dfs_ext.groupby(level=1, sort=False).std(ddof=0) ## STDS ddof=1

In [ ]:
mean_metrics

In [ ]:
std_metrics

In [ ]:
# all_dfs.to_csv('mae_110225_Spectraratio75_spS20_Emb128_0cos500epochs_60all_seedsRun_LinearProbing_FullRange_val.csv') #val ext
# all_dfs_ext.to_csv('mae_110225_Spectraratio75_spS20_Emb128_0cos500epochs_60all_seedsRun_LinearProbing_FullRange_ext.csv') #val ext

# all_dfs.to_csv('mae_110225_Spectraratio75_spS20_Emb128_0cos500epochs_80all_seedsRun_FineTuning_FullRange_val.csv')
# all_dfs_ext.to_csv('mae_110225_Spectraratio75_spS20_Emb128_0cos500epochs_80all_seedsRun_FineTuning_FullRange_ext.csv')


# all_dfs.to_csv('mae_110225_Spectraratio75_spS20_Emb128_0cos500epochs_60all_seedsRun_LinearProbing_HalflRange_val.csv') #val ext
# all_dfs_ext.to_csv('mae_110225_Spectraratio75_spS20_Emb128_0cos500epochs_60all_seedsRun_LinearProbing_HalflRange_ext.csv') #val ext

# all_dfs.to_csv('mae_110225_Spectraratio75_spS20_Emb128_0cos500epochs_60all_seedsRun_FineTuning_HalflRange_val.csv')
# all_dfs_ext.to_csv('mae_110225_Spectraratio75_spS20_Emb128_0cos500epochs_60all_seedsRun_FineTuning_HalflRange_ext.csv')

# all_dfs.to_csv('mae_110225_Spectraratio75_spS20_Emb128_0cos500epochs_20ecosis_seedsRun_LinearProbing_FullRange_val.csv') #val ext
# all_dfs_ext.to_csv('mae_110225_Spectraratio75_spS20_Emb128_0cos500epochs_20ecosis_seedsRun_LinearProbing_FullRange_ext.csv') #val ext

# all_dfs.to_csv('mae_110225_Spectraratio75_spS20_Emb128_0cos500epochs_20ecosis_seedsRun_FineTuning_FullRange_val.csv')
# all_dfs_ext.to_csv('mae_110225_Spectraratio75_spS20_Emb128_0cos500epochs_20ecosis_seedsRun_FineTuning_FullRange_ext.csv')

In [1]:
from transformation_utils import *
from utils_all import *
from utils_data import *

from MAE.utils_mae import *
from MAE.trainer_mae import *
from MAE.MAE_1D import *
from MAE.multi_trait import *
from MAE.trainer_trait import *

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import gc

##################
d = 10
h = 16
w_loss = 1

SeqSize = 20


path_model_mae = '/home/mila/e/eya.cherif/scratch/mae/checkpoints_MAE_250328_0608_Training_SeqSize{}_allUNlabels_155/best_model.h5'.format(SeqSize)
path_data_lb = '/home/mila/e/eya.cherif/Gan_project_test/49_all_lb_prosailPro.csv'
ls_tr = ["cab", "cw", "cm", "LAI", "cp", "cbc", "car", "anth"]

percentage_tr = 0.4
batch_size = 256


settings_dict_mae = {
    'learning_rate': 1e-3,
    'weight_decay': 0,
    'load_model_path': path_model_mae, #path_model_mae load_model_path
    'should_save_models': True,
    'skip_completed_experiment': True,
    'number_of_data_workers': 0,
    'w_loss' : w_loss,
    
    'mask_ratio': 0.75,
    'n_bands': 1720,
    'seq_size': SeqSize,
    'in_chans': 1,
    'embed_dim': 128,
    'depth': d,
    'num_heads': h,
    'decoder_embed_dim': 128,
    'decoder_depth': 6,
    'decoder_num_heads': 4,
    'mlp_ratio': 4.0,
    'norm_layer': nn.LayerNorm,
    'cls_token': False,
    'device': torch.device("cuda" if torch.cuda.is_available() else "cpu")
}


sets = Settings()
sets.update_from_dict(settings_dict_mae)

test_mae = Trainer(sets)

test_mae.model_setup()
test_mae.model = torch.load(settings_dict_mae['load_model_path']) 


########## Trait regression ####
################ Data ###############
db_lb_all = pd.read_csv(path_data_lb, low_memory=False).drop(['Unnamed: 0'], axis=1)   

### external
groups = db_lb_all.groupby('dataset')

val_ext_idx = list(groups.get_group(32).index)+list(groups.get_group(3).index)
samples_val_ext = db_lb_all.loc[val_ext_idx,:]
db_lb_all.drop(val_ext_idx, inplace=True)

X_labeled, y_labeled, _ = data_prep_db(db_lb_all, ls_tr, weight_sample=True)
metadata = db_lb_all.iloc[:, :8]  # The metadata (dataset of origin)


red_ed = X_labeled.loc[:,750]
red_end = X_labeled.loc[:,1300]
red1000_ = X_labeled.loc[:,1000]

idx = X_labeled[(red_end>red1000_) & (red_ed>red1000_)].index

if(len(idx)>0):
    # X_labeled.loc[idx,:].T.plot(legend=False)
    X_labeled.drop(idx, inplace=True)
    y_labeled.drop(idx, inplace=True)
    metadata.drop(idx, inplace=True)


# Split labeled data into train (80%), validation (10%), and test (10%)
fr_sup, val_x= train_test_split(X_labeled, test_size=0.2, stratify=metadata.dataset, random_state=300)

y_sup = y_labeled.loc[fr_sup.index,:]
meta_train = metadata.loc[fr_sup.index,:]

val_y = y_labeled.loc[val_x.index,:]
meta_val = metadata.loc[val_x.index,:]

if(percentage_tr<1):
    fr_sup, _= train_test_split(fr_sup, test_size=1-percentage_tr, stratify=meta_train.dataset, random_state=300)
    
    y_sup = y_sup.loc[fr_sup.index,:]
    meta_train = meta_train.loc[fr_sup.index,:]

# ########## Scaler ###
scaler_list = None
scaler_model = save_scaler(y_sup, standardize=True, scale=True)

###### Create the dataset#################

train_dataset = SpectraDataset(fr_sup, y_sup, meta_train, augmentation=True, aug_prob=0.6)
# Define DataLoader with the custom collate function for fair upsampling
train_dataset_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

#####
test_dataset = SpectraDataset(X_train=val_x, y_train=val_y, meta_train=meta_val, augmentation=False)
# Create DataLoader for the test dataset
valid_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

###########
# DB50_all_F_SHIFT.csv
path_data_lb_ext = '/home/mila/e/eya.cherif/Gan_project_test/DB50_all_F_SHIFT.csv'
db_lb_all_ext = pd.read_csv(path_data_lb_ext, low_memory=False).drop(['Unnamed: 0'], axis=1)   

ext_all = pd.concat([samples_val_ext,db_lb_all_ext])

ext_val_x = feature_preparation(ext_all.loc[:, '400':'2500']).loc[:, 400:2450] #ext_all samples_val_ext
ext_val_y = ext_all[ls_tr] # ext_all  samples_val_ext

# Validation dataset
x_p_val = torch.tensor(ext_val_x.values, dtype=torch.float)#.unsqueeze(dim=1) ##ext_val_x.values scaler_spectra.transform(ext_val_x.iloc[:,:-1])
lb_p_val = torch.tensor(ext_val_y.values,dtype=torch.float)

external_dataset = TensorDataset(x_p_val, lb_p_val) # x_p_val
valid_ext_loader = DataLoader(external_dataset, batch_size=batch_size, shuffle=False)

Encoder has 2006912 parameters.
Decoder has 1220116 parameters.


In [2]:
fr_sup.shape

(1803, 1721)

In [3]:
settings_dict = {
        # 'epochs': 50,
        'train_loader': train_dataset_loader,
        'valid_loader': valid_loader,
        # 'checkpoint_dir': checkpoint_dir,
        'batch_size': batch_size,
        'learning_rate': 1e-3,
        'weight_decay': 1e-4,
        'pretrained_model': test_mae.model,#model,
        'freeze': False,
        'early_stop': False,
        'patience': 10,
        'scaler_model': scaler_model,
    }

##########

# SEED = 155
# Create an empty dictionary
eval_scores = {}
ext_scores = {}

results_DS = []
results_DS_ext = []

for SEED in [155, 381, 187]: #, 381, 187

    # Python random seed
    random.seed(SEED)
    
    # Numpy seed
    np.random.seed(SEED)
    
    # PyTorch seed
    torch.manual_seed(SEED)
    
    # If using CUDA
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)  # If using multi-GPU
    
    # Ensuring deterministic behavior
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # val_r2, val_nrmse = downstream(test_mae.model, FT = False, num_epochs=200) ##downstream(pretrained_model, FT = False, num_epochs=200)
    val_r2, val_nrmse, ext_r2, ext_nrmse = downstream(test_mae.model, FT = False, num_epochs=200) ##downstream(pretrained_model, FT = False, num_epochs=200)
    results_DS.append((SEED, val_r2, val_nrmse))
    results_DS_ext.append((SEED, ext_r2, ext_nrmse))
    
    gc.collect()
    torch.cuda.empty_cache()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 16.88it/s]


epoch: 1 Training Loss: 297.300810 	 Validation Loss: 225.149966, tr_r2: -0.050178, val_r2: 0.118653 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 12.44it/s]


epoch: 2 Training Loss: 233.907113 	 Validation Loss: 191.860168, tr_r2: 0.213493, val_r2: 0.260404 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 24.63it/s]


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 13.73it/s]


epoch: 199 Training Loss: 109.665444 	 Validation Loss: 108.034196, tr_r2: 0.665459, val_r2: 0.600826 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:00<00:00, 24.30it/s]


epoch: 200 Training Loss: 110.183513 	 Validation Loss: 107.513558, tr_r2: 0.664663, val_r2: 0.604174 


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 9/9 [00:00<00:00, 21.75it/s]


In [ ]:
# ####
# rpd metric!!!
# adjusted R2

In [4]:
all_dfs = pd.DataFrame(results_DS)
all_dfs_ext = pd.DataFrame(results_DS_ext)

df_aggregated = pd.DataFrame({
    1: np.mean(all_dfs[1]),  # Concatenate all lists in column 1
    2: np.mean(all_dfs[2]),   # Concatenate all lists in column 2
    3: np.mean(all_dfs_ext[1]),  # Concatenate all lists in column 1
    4: np.mean(all_dfs_ext[2]),   # Concatenate all lists in column 2
})


df_aggregated

,1,2,3,4
0,0.494974,17.728626,0.539423,14.458517
1,0.538783,14.446641,0.232277,19.394103
2,0.613519,11.516132,0.742574,11.864938
3,0.523114,18.120197,0.394587,19.305491
4,0.472589,12.645172,0.215657,20.991588
5,0.651585,11.120898,0.299577,22.050206
6,0.452584,14.951458,-0.183901,21.513551
7,0.268630,19.061042,0.141135,21.298306


In [5]:
df_aggregated.quantile(0.5)

1     0.509044
2    14.699049
3     0.265927
4    20.192845
Name: 0.5, dtype: float64

In [6]:
results_DS

[(155,
  array([0.49317209, 0.52944352, 0.61376505, 0.51275415, 0.47091366,
         0.6536707 , 0.44852822, 0.27297229]),
  array([17.76025388, 14.59264563, 11.51312256, 18.31654968, 12.66541075,
         11.08781058, 15.00679669, 19.00482258])),
 (381,
  array([0.49432547, 0.54734236, 0.60328538, 0.5308616 , 0.46689835,
         0.6451278 , 0.45326004, 0.25472166]),
  array([17.74003392, 14.31242033, 11.66826878, 17.97297927, 12.71337955,
         11.22372839, 14.942277  , 19.24188319])),
 (187,
  array([0.49742449, 0.53956461, 0.62350654, 0.5257272 , 0.47995512,
         0.6559567 , 0.45596277, 0.27819638]),
  array([17.68559043, 14.43485702, 11.36700485, 18.07106257, 12.5567259 ,
         11.05115647, 14.90529903, 18.93642034]))]

In [7]:
results_DS_ext

[(155,
  array([ 0.53983724,  0.20221362,  0.74551166,  0.35535782,  0.21511095,
          0.31735637, -0.21672557,  0.14238388]),
  array([14.4520151 , 19.77312214, 11.80061611, 19.92706731, 20.99996881,
         21.77206107, 21.81083826, 21.28354066])),
 (381,
  array([ 0.5397295 ,  0.26651547,  0.7257679 ,  0.42694825,  0.23532641,
          0.26378646, -0.17358619,  0.15779282]),
  array([14.45370739, 18.95952192, 12.24982579, 18.78801732, 20.72776926,
         22.61020137, 21.42069255, 21.09147203])),
 (187,
  array([ 0.53870223,  0.22810115,  0.75644383,  0.40145443,  0.19653454,
          0.31758872, -0.16139272,  0.12322973]),
  array([14.46982744, 19.44966459, 11.5443715 , 19.20138901, 21.2470249 ,
         21.768356  , 21.30912213, 21.51990399]))]